In [176]:
import json
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import os
import numpy as np
import csv as csv
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split                                 

In [3]:
# data=pd.read_csv('..\Datasets/HpState/shovel_39_data_fixed.csv')
# raw_output=
# data

,Unnamed: 0,capture_time,heading,date_time,sequence_number,roll,pitch,height,latitude,longitude,time_without_microsecond,id
0,0,25:00.0,-17.425865,24:51.7,677026,0.516578,-48.354008,1713.492337,49.726845,-114.798926,4/14/2021 12:24,0
1,1,25:00.1,-17.478781,24:51.8,677027,0.584451,-47.634618,1713.599990,49.726847,-114.798926,4/14/2021 12:24,1
2,2,25:00.2,-17.479210,24:51.9,677028,0.638639,-46.931638,1713.698169,49.726848,-114.798927,4/14/2021 12:24,2
3,3,25:00.3,-17.544945,24:52.0,677029,0.661536,-46.279095,1713.793497,49.726849,-114.798928,4/14/2021 12:24,3
4,4,25:00.4,-17.618170,24:52.1,677030,0.709778,-45.542828,1713.901479,49.726850,-114.798929,4/14/2021 12:24,4
...,...,...,...,...,...,...,...,...,...,...,...,...
5661,5661,34:34.2,-136.196198,34:25.9,682687,0.985247,7.438396,1725.821861,49.726587,-114.799174,4/14/2021 12:34,5661
5662,5662,34:34.3,-136.644287,34:26.0,682688,0.938363,7.981765,1725.954983,49.726587,-114.799172,4/14/2021 12:34,5662
5663,5663,34:34.4,-137.088745,34:26.1,682689,0.893205,8.530322,1726.086585,49.726586,-114.799170,4/14/2021 12:34,5663
5664,5664,34:34.5,-137.479034,34:26.2,682690,0.856316,9.090152,1726.211577,49.726586,-114.799168,4/14/2021 12:34,5664


In [122]:
data_address=["..\Datasets/HpState/data-1620089521423.csv"]#,"..\Datasets/HpState/2021-06-03-5AM-8AM.csv"]
label_address=["..\Datasets/HpState/output.xlsx"]#,"..\Datasets/HpState/labels_2_updated.csv"]
#test=pd.read_excel('..\Datasets/HpState/output.xlsx')
temp_df=pd.read_csv(data_address[0])[["capture_time","body"]]
temp_df

labels_list=pd.read_excel("..\Datasets/HpState/output (3).xls",engine = 'xlrd')[["start","end","state"]]

labels_list

# temp_dict={}
# for key in ["pitch","roll","latitude","longitude","heading","height","date_time","parts"]:
#     temp_dict[key]=[]

,start,end,state
0,2021-04-14 06:24:55.746,2021-04-14 06:25:01.879,swing to dig
1,2021-04-14 06:25:01.879,2021-04-14 06:25:14.578,dig
2,2021-04-14 06:25:14.578,2021-04-14 06:25:24.578,swing to dump
3,2021-04-14 06:25:24.578,2021-04-14 06:25:29.012,dump
4,2021-04-14 06:25:29.012,2021-04-14 06:25:39.879,swing to dig
...,...,...,...
1214,2021-04-14 12:34:28.237,2021-04-14 12:34:48.804,idle
1215,2021-04-14 12:34:48.804,2021-04-14 12:35:32.503,propel
1216,2021-04-14 12:35:32.503,2021-04-14 12:35:38.937,swing to dig
1217,2021-04-14 12:35:38.937,2021-04-14 12:35:58.237,cleaning


In [125]:
data_list=[]


for i in range(len(data_address)):
    temp_df=pd.read_csv(data_address[i])[["capture_time","body"]]
    temp_df["parts"]=i  #[i for j in range(temp_df.shape[0])]
    data_list.append(temp_df)
    
data=pd.concat(data_list,axis=0)
labels=labels_list.copy()
data=data.reset_index(drop=True)
labels=labels.reset_index(drop=True)

In [126]:
df = pd.DataFrame.from_records(data.body.apply(lambda x : json.loads(x)))
df2 = pd.DataFrame.from_records(df.GET_PIN_POR_HP_BUCKET_STRUCT_TAG)
df3 = pd.DataFrame.from_records(df2.crowd_orientation)
df4 = pd.DataFrame.from_records(df2.point_of_interest_global)
if "update_interval_ms" in df2.keys():
    df_ = pd.concat([df2[['heading', 'date_time','update_interval_ms']], df3,df4], axis=1)
else:
    df_ = pd.concat([data.capture_time,df2[['date_time','heading']], df3,df4], axis=1)
print("data collection from json:\r\n")
second_shift=[7,0]
micro_second_shift=[879000,0]

# data=sync_date_time(data,second_shift,micro_second_shift)
# def sync_date_time(data,second_shift,micro_second_shift):
#df=df_.copy()
#df_list=[]
#for i in df.parts.unique():
#    temp_df=df[df.parts==i]
df_["date_time"]= pd.to_datetime(df_["date_time"]).apply(lambda x:x+datetime.timedelta(seconds=7,microseconds=879000))
#df_list.append(temp_df)
#df_= pd.concat(df_list,axis=0).reset_index(drop=True)
df_


data collection from json:



,capture_time,date_time,heading,roll,pitch,height,latitude,longitude
0,25:00.0,2021-04-14 12:25:07.914,-17.425865,0.516578,-48.354008,1713.492337,49.726845,-114.798926
1,25:00.1,2021-04-14 12:25:08.011,-17.478781,0.584451,-47.634618,1713.599990,49.726847,-114.798926
2,25:00.2,2021-04-14 12:25:08.109,-17.479210,0.638639,-46.931638,1713.698169,49.726848,-114.798927
3,25:00.3,2021-04-14 12:25:08.207,-17.544945,0.661536,-46.279095,1713.793497,49.726849,-114.798928
4,25:00.4,2021-04-14 12:25:08.319,-17.618170,0.709778,-45.542828,1713.901479,49.726850,-114.798929
...,...,...,...,...,...,...,...,...
16173,52:20.6,2021-04-14 12:52:28.496,-39.155704,1.601719,10.100700,1727.032807,49.726840,-114.799130
16174,52:20.7,2021-04-14 12:52:28.592,-39.167282,1.595665,10.100829,1727.025626,49.726840,-114.799130
16175,52:20.8,2021-04-14 12:52:28.692,-39.176727,1.592128,10.087653,1727.024810,49.726840,-114.799131
16176,52:20.9,2021-04-14 12:52:28.803,-39.188408,-5.375136,69.049701,1735.519134,49.726756,-114.799021


In [143]:
labels_list['state']

0        swing to dig
1                 dig
2       swing to dump
3                dump
4        swing to dig
            ...      
1214             idle
1215           propel
1216     swing to dig
1217         cleaning
1218           propel
Name: state, Length: 1219, dtype: object

In [167]:
df=df_
lbs=labels_list.copy()


lbs["start"] = lbs["start"].apply(lambda x : pd.to_datetime(x)+datetime.timedelta(hours=6))  # WHY 6hours were added
lbs["end"] = lbs["end"].apply(lambda x : pd.to_datetime(x)+datetime.timedelta(hours=6))

temp=[]
df.shape[0]
for i in range(df.shape[0]):
    temp_data=lbs[lbs["start"]<=df.iloc[i]["date_time"]]
    temp_data=temp_data[temp_data["end"]>df.iloc[i]["date_time"]]
    # temp=temp_data["state"]
#     t[i]=temp
    if temp_data.shape[0]>0:
        temp.append(temp_data.iloc[0]["state"]) # WHY ILOC[0]??????????????????
    else:
        temp.append(np.nan)
df["state"]=temp

In [168]:
df["state"].unique()

array(['dig', 'swing to dump', 'dump', 'swing to dig', 'cleaning',
       'wait for truck', 'idle', 'other', 'propel'], dtype=object)

In [174]:
df_.to_csv("..\Datasets/HpState/sych_data_with_label.csv")

In [175]:
correlation=df_.corr()
correlation

,heading,roll,pitch,height,latitude,longitude
heading,1.000000,-0.096029,0.008396,0.019551,-0.091903,0.835234
roll,-0.096029,1.000000,-0.335356,-0.281179,0.060792,-0.325707
pitch,0.008396,-0.335356,1.000000,0.980463,-0.061202,0.106319
height,0.019551,-0.281179,0.980463,1.000000,-0.031314,0.110608
latitude,-0.091903,0.060792,-0.061202,-0.031314,1.000000,-0.270026
longitude,0.835234,-0.325707,0.106319,0.110608,-0.270026,1.000000


In [184]:
# correlation/coefficient heading and longitude
X=df_.iloc[:,2]
Y=df_.iloc[:,-2]





In [186]:
# regressoin longitude and heading
lr_heading_longitude=LinearRegression()
lr_heading_longitude.fit(X,Y)

ValueError: Expected 2D array, got 1D array instead:
array=[-17.42586517 -17.47878075 -17.4792099  ... -39.17672729 -39.1884079
 -39.20661545].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [165]:

# getattr( t[0],'state')
# r=lbs[lbs["start"]<=df.iloc[0]["date_time"]]["state"]
# s=pd.Series(r)
# list(s)


# count=0
# for x in range(len(t)):
#     s=pd.Series(t[x])
#     if len(list(s))>1:
#         print(t[x])
#         count+=1
# print(count)


0


In [164]:
s=pd.Series(t[x])
len(list(s))

1

In [ ]:


def get_numbers(x):
    temp=pd.Series(x)
    temp=temp[temp!=0]
    return list(temp)

# def extract_dataframe(data):
#     df=data.copy()
#     temp_dict={}
#     part=df.iloc[0]["parts"]

#     for key in ["pitch","roll","latitude","longitude","heading","height","date_time","parts"]:
#         temp_dict[key]=[]
#     for i in range(df.shape[0]):
#         for key in ["pitch","roll","latitude","longitude","heading","height"]:
#             temp=pd.Series(df.iloc[i][key])     #???????????????????????????????????????????????????????
#             temp=temp[temp!=0]             
#             temp_dict[key].extend(list(temp))
#         l=len(get_numbers(df.iloc[i]["heading"]))
#         update_interval=df.iloc[i]["update_interval_ms"]*1000
#         temp_date_time=[]
#         for j in range(l):
#             temp_date_time.append(df.iloc[i]["date_time"]+datetime.timedelta(microseconds=int(update_interval*j)))
#         temp_dict["date_time"].extend(temp_date_time)
#     temp_dict["parts"]=[part for i in range(len(temp_dict["date_time"]))]
#     return pd.DataFrame(temp_dict)



#print("transforming...")
#data=transform(data)
#parts=data.parts.unique()
# temp_df=[]
# #    for i in parts:
# df=data[data["parts"]==0]
# if type(df.iloc[0]["pitch"])==list:
#     print(f"transforming part 0")
#     temp_df.append(extract_dataframe(df[df["parts"]==0]))
# else:
#     temp_df.append(df.drop(["update_interval_ms"],axis=1,errors="ignore"))
# return pd.concat(temp_df,axis=0).reset_index(drop=True).astype({"pitch":float,"roll":float,"latitude":float,"longitude":float,"heading":float,"height":float,"parts":float})



# #data["id"]=data.index
# if denoising:
#     print("denoising")
#     data=denoise0(data)
# print("averaging")
# data=data.set_index(data["date_time"])
# def create_data_averaged_over_seconds(data):
    # df=data.copy()
    # parts=df["parts"].unique()
    # temp_list=[]
    # for i in parts:
    #     print(i)
    #     temp_df=df[df["parts"]==i].resample("0.5S").mean()
    #     temp_df=temp_df.reset_index()
    #     temp_df=temp_df.fillna(method='ffill')
    #     temp_list.append(temp_df)
    # df=pd.concat(temp_list,axis=0)
    # df=df.reset_index(drop=True)
    # return df

# data_averaged_over_seconds=create_data_averaged_over_seconds(data)
# data_averaged_over_seconds.head()
# # data=data.reset_index(drop=True)


# def upsample(dataframe):
#     dict_of_dataframes={}
#     size=[]
#     for key in dataframe["state"].unique():
#         if dataframe[dataframe["state"]==key].shape[0]>0:
#             dict_of_dataframes[key]=dataframe[dataframe["state"]==key]
#             size.append(dict_of_dataframes[key].shape[0])
#     m=np.max(size)
#     dataframes=[]
#     for key in dict_of_dataframes.keys():
#         if m/dict_of_dataframes[key].shape[0]<8:
#             for i in range(int(m/dict_of_dataframes[key].shape[0])):
#                  dataframes.append(dict_of_dataframes[key])
#                  dataframes.append(dict_of_dataframes[key].sample(m%dict_of_dataframes[key].shape[0]))
#     return pd.concat(dataframes,axis=0)

# def apply_labels(data,labels,window_size=10,up=False):
#     df=data.copy()
#     lbs=labels.copy()
#     lbs["start"] = lbs["start"].apply(lambda x : get_date_time(str(x)))
#     lbs["end"] = lbs["end"].apply(lambda x : get_date_time(str(x)))
#     temp=[]
#     for i in tqdm.tqdm(range(df.shape[0])):
#         temp_data=lbs[lbs["start"]<=df.iloc[i]["date_time"]]
#         temp_data=temp_data[temp_data["end"]>df.iloc[i]["date_time"]]
#         if temp_data.shape[0]>0:
#             temp.append(temp_data.iloc[0]["state"]) 
#         else:
#             temp.append(np.nan)
#     df["state"]=temp
#     if up:
#         return upsample(df)
#     else:
#         return df